# MRI Brain Tumour Classifier based on DenseNet Transfer learning

References:
- [Reference 1](https://www.kaggle.com/code/abdoghazala/brain-tumor-detection-classification-cnn-97-6)


In [ ]:
# Import packages
import os
import random
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from colorama import Fore

import matplotlib.pyplot as plt
from utils import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
import tensorflow.keras.layers as tfl
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications import DenseNet121

In [ ]:
# Preprocessing

IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# Train
training_set = tf.keras.utils.image_dataset_from_directory(
    DATASET_FOLDER,
    validation_split = 0.2,
    subset="training",
    shuffle=True,
    seed=19032024,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE,
    label_mode = 'categorical'
)
# Validation
validate_set = tf.keras.utils.image_dataset_from_directory(
    DATASET_FOLDER,
    validation_split = 0.2,
    subset="validation",
    shuffle=True,
    seed=19032024,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE,
    label_mode = 'categorical'

)

test_set = tf.keras.utils.image_dataset_from_directory(
    DATASET_FOLDER,
    shuffle=False,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE,
    label_mode = 'categorical'
)

# Prefetch the train_dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = training_set.prefetch(buffer_size=AUTOTUNE)
validation_dataset = training_set.prefetch(buffer_size=AUTOTUNE)


In [ ]:
class_names = training_set.class_names
print(class_names)
class_names = validate_set.class_names
print(class_names)

In [ ]:
# View images of dataset
class_names = test_set.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[np.argmax(labels[i])] , c = 'blue' , size = 10)
    plt.axis("off")

plt.figure(figsize=(10, 10))
for images, labels in validation_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[np.argmax(labels[i])] , c = 'blue' , size = 10)
    plt.axis("off")

Models

In [ ]:
# Hyperparameters
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH) + (3,)
LOSS = keras.losses.CategoricalCrossentropy()
OPTIMIZER = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
SEED = 42

In [ ]:
# # callbacks for the models
# def get_callbacks (model_name):
#   callbacks = []
#   checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath= f'{model_name}', monitor = 'val_loss', verbose = 1 ,
#                                                     mode = 'min', save_best_only=True, save_freq='epoch')
#   callbacks.append(checkpoint)
#   rlr = tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.2, patience=5, mode='auto', min_lr=0.0)
#   callbacks.append(rlr)
#   earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 7)
#   callbacks.append(earlystop)

#   return callbacks

In [ ]:
# function for plot resultes

def plot():
   pd.DataFrame(history.history)[['categorical_accuracy','val_categorical_accuracy','loss','val_loss']].plot( figsize=(7, 5), xlim=[0, 9], ylim=[0, 1], grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])
   plt.show()

# for make dataframe for densenet model
model_name= 'DenseNet121'
CategoricalAccuracy= []
losses= []

## Densenet Portion

In [ ]:
def preprocess_data(image, label):
    return preprocess_input(image), label

train_dataset_densenet = train_dataset.map(preprocess_data, num_parallel_calls=AUTOTUNE)
validation_dataset_densenet = validation_dataset.map(preprocess_data, num_parallel_calls=AUTOTUNE)
test_dataset_densenet = test_set.map(preprocess_data, num_parallel_calls=AUTOTUNE)

dense_model = DenseNet121(weights='imagenet', include_top=False, input_shape= IMG_SHAPE , classes = 4)
dense_model.trainable = False

x = dense_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=dense_model.input, outputs=predictions)

model.compile(optimizer= OPTIMIZER, loss= LOSS, metrics=[keras.metrics.CategoricalAccuracy() , keras.metrics.Recall()])

model.summary()

In [ ]:
# callbacks = get_callbacks('DenseNet121')

history = model.fit(
    train_dataset_densenet,
    epochs=10,
    validation_data=validation_dataset_densenet,
)

In [ ]:
plot()

# Test metrics
loss, categorical_accuracy, recall = model.evaluate(test_dataset_densenet)
CategoricalAccuracy.append(categorical_accuracy)
losses.append(loss)

In [ ]:
# Saving the model

# Create the directory if it doesn't exist
save_dir = 'model/'
os.makedirs(save_dir, exist_ok=True)

model.save(os.path.join(save_dir, 'densenet121.keras'))


## Midnight Maximum Fine Tuning

In [ ]:
densenet_tuning = keras.models.load_model('./model/densenet121.keras')

In [ ]:
for layer in densenet_tuning.layers[:17]:
    layer.trainable = False

# Set last 5 layers to be trainable
for layer in densenet_tuning.layers[17:]:
    layer.trainable = True

history = densenet_tuning.fit(
    train_dataset_densenet,
    epochs = 10,
    validation_data = validation_dataset_densenet,
)

In [ ]:
loss, categorical_accuracy = densenet_tuning.evaluate(test_dataset_densenet)

In [ ]:
# For model evaluation in confusion matrix
y_true = np.concatenate([ y for _ , y in test_set] , axis = 0)

In [ ]:
y_true = np.argmax(y_true, axis=1)
y_pred = densenet_tuning.predict(test_dataset_densenet)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report , ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_true , y_pred)
plt.title("Confusion Matrix", fontname = "monospace", fontsize = 15, weight = "bold")
plt.show()

In [ ]:
print("\nClassification Report:\n")
print(Fore.BLUE + classification_report(y_true, y_pred, target_names = class_names, digits= 4))